In [69]:
import os

import torch
import torch.nn as nn
from torch.autograd import Variable

import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torchvision.utils as vutil

import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# os.getcwd()
os.chdir('/Users/dengyi/Desktop/program_instances/pytorch_practice')

In [70]:
image_size = 28
input_dim = 100
num_channels = 1
num_features = 64
batch_size = 64

In [71]:
use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
itype = torch.cuda.LongTensor if use_cuda else torch.LongTensor

In [72]:
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(),
                            download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

In [73]:
indices = range(len(test_dataset))
indices_val = indices[:5000]
indices_test = indices[5000:]

In [74]:
sampler_val = torch.utils.data.sampler.SubsetRandomSampler(indices_val)
sampler_test = torch.utils.data.sampler.SubsetRandomSampler(indices_test)

validation_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, sampler=sampler_val)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, sampler=sampler_test)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [75]:
def make_show(img):
    img = img.data.expand(batch_size, 3, image_size, image_size)
    return img

def imshow(inp, title=None, ax=None):
    """Imshow for Tensor."""
    if inp.size()[0] > 1:
        inp = inp.numpy.transpose((1, 2, 0))
    else:
        inp = inp[0].numpy()

    min_value = np.amin(inp)
    max_value = np.amax(inp)
    if max_value > min_value:
        inp = (inp - min_value) / (max_value - min_value)
    ax.imshow(inp)
    if title is not None:
        ax.set_title(title)


In [76]:
class ModelG(nn.Module):
    def __init__(self):
        super(ModelG, self).__init__()
        self.model = nn.Sequential()
        self.model.add_module('deconv1', nn.ConvTranspose2d(input_dim, num_features*2,
                                                            5, 2, 0, bias=False))
        self.model.add_module('bnorm1', nn.BatchNorm2d(num_features*2))
        self.model.add_module('relu1', nn.ReLU(True))
        self.model.add_module('deconv2', nn.ConvTranspose2d(num_features*2, num_features, 
                                                            5, 2, 0, bias=False))
        self.model.add_module('bnorm2', nn.BatchNorm2d(num_features))
        self.model.add_module('relu2', nn.ReLU(True))
        self.model.add_module('deconv3', nn.ConvTranspose2d(num_features, num_channels, 
                                                            4, 2, 0, bias=False))
        self.model.add_module('sigmoid', nn.Sigmoid())

    def forward(self, input):
        output = input
        for name, module in self.model.named_children():
            output = module(output)
        return output

In [77]:
def weight_init(m):
    class_name = m.__class__.__name__
    if class_name.find('conv') != -1:
        m.weight.data.normal_(0, 0.02)
    if class_name.find('norm') != -1:
        m.weight.data.normal_(1.0, 0.02)

In [78]:
net = ModelG()
net = net.cuda() if use_cuda else net

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

In [79]:
samples = np.random.choice(10, batch_size)
samples = Variable(torch.from_numpy(samples).type(dtype))
samples.resize_(batch_size, 1, 1, 1)
samples = Variable(samples.data.expand(batch_size, input_dim, 1, 1))
samples = samples.cuda() if use_cuda else samples

def save_evaluation_samples(netModel, epoch, save_path='gan'):
    save_path = save_path.strip()
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    fake_u = netModel(samples)
    fake_u = fake_u.cuda() if use_cuda else fake_u
    img = make_show(fake_u)
    vutil.save_image(img, os.path.join(save_path, 'fake_{}.png'.format(epoch)))

In [80]:
def train_ModelG(target, data):
    if use_cuda:
        target, data = target.cuda(), data.cuda()
    print(data.size())
    data = data.type(dtype)
    
    data = data.reshape(data.size()[0]*data.size()[2]*data.size()[3], 1, 1, 1)
    data = data.expand(data.size()[0], input_dim, 1, 1)
    
    net.train()
    output = net(data)
    loss = criterion(output, target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if use_cuda:
        loss = loss.cuda()
    return loss

In [81]:
def evaluate_ModelG():
    net.eval()
    val_loss = []

    idx = 0
    for (data, target) in validation_loader:
        target, data = Variable(data), Variable(target)
        idx += 1
        if use_cuda:
            target, data = target.cuda(), data.cuda()
        data = data.type(dtype)
        data = data.reshape(data.size()[0]*data.size()[2]*data.size()[3], 1, 1, 1)
        data = data.expand(data.size()[0], input_dim, 1, 1)
        output = net(data)
        loss = criterion(output, target)
        if use_cuda:
            loss = loss.cuda()
        val_loss.append(loss.data.numpy())
    return val_loss

#### 训练模型

In [83]:
print("Initialized!")

step = 0
num_epoches = 1
record = []

for epoch in range(num_epoches):
    train_loss = []

    for batch_idx, (data, target) in enumerate(train_loader):
        target, data = Variable(target), Variable(data)
        loss = train_ModelG(target, data)
        train_loss.append(loss.data.numpy())
        step += 1

        if step % 100 == 0:
            val_loss = evaluate_ModelG()
            print('训练周期：{} [{}/{}({:.0f}%)]\t训练数据 Loss: {:.6f}\t校验数据 Loss：{:.6f}'.format(
                epoch, batch_idx*batch_size, len(train_loader.dataset), 100.*batch_idx/len(train_loader), 
                np.mean(train_loss), np.mean(val_loss)
            ))
            record.append([np.mean(train_loss), np.mean(val_loss)])

        save_evaluation_samples(net, epoch, 'MSE')

Initialized!
torch.Size([64, 1, 28, 28])


/Users/dengyi/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([50176, 1, 28, 28])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: The size of tensor a (28) must match the size of tensor b (64) at non-singleton dimension 3

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot([i[0] for i in record], label='Training')
plt.plot([i[1] for i in record], label='Validation')
plt.xlabel('Batches')
plt.ylabel('Loss')
plt.legend()